In [108]:
from numpy import linalg as LA
from numpy import array, identity, ones, nonzero, zeros, argmax
import networkx as nx

file = "top250movies.txt"

In [109]:
try:
    with open(file, "r", encoding = "utf-8") as f:
        actorDict = {}
        lineList = f.readlines()
        for line in lineList:
            line = line.strip() # remove leading and trailing whitespace
            line = line.split("/") # split line into list of strings
            actorDict[line[0]] = line[1:] 
except FileNotFoundError:
    raise IOError(f"Error: Could not find file {file}.")
except ValueError:
    raise ValueError(f"Error: Invalid data in file {file}.")

In [110]:
links = sorted(actorDict)
n = len(links)
G = zeros((n,n))

In [111]:
actorGraph = nx.MultiDiGraph()
for actors in actorDict.values():
    for i, actor in enumerate(actors):
        if i > 0:
            cheaperActor = actors[i]
            actorGraph.add_edge(cheaperActor, actor)
           
directedActorGraph = nx.DiGraph()
for actors in actorDict.values():
    for i, actor in enumerate(actors):
        if i > 0:
            cheaperActor = actors[i]
            directedActorGraph.add_edge(cheaperActor, actor, weight=1)

In [112]:
# Compute PageRank
netw = nx.pagerank(directedActorGraph, alpha=0.85)

# Sort PageRank
sortedPageRank = sorted(netw.items(), key=lambda x: x[1], reverse=True)

for actor in sortedPageRank[:10]:
    print(actor[0])

Morgan Freeman
Bob Gunton
William Sadler
Clancy Brown
Gil Bellows
Mark Rolston
James Whitmore
Jeffrey DeMunn
Larry Brandenburg
Neil Giuntoli
